## SageMaker environment variables
SageMaker passes all environment variables - including hyperparameters - to the training scripts as command-line arguments.  Your script can parse these arguments.

Arguments include hyperparamers, locations to training/validation datasets, and number of cpus/gpus, etc. 

A full list of environment variables an be found on the [SageMaker Container](https://github.com/aws/sagemaker-containers#important-environment-variables) GitHub repository.

Here is an example of how to parse the arguments in your Python script:

```
parser = argparse.ArgumentParser()

# Hyper-parameters
parser.add_argument('--epochs',        type=int,   default=15)
parser.add_argument('--learning-rate', type=float, default=0.001)
parser.add_argument('--batch-size',    type=int,   default=256)
parser.add_argument('--weight-decay',  type=float, default=2e-4)
parser.add_argument('--momentum',      type=float, default='0.9')
parser.add_argument('--optimizer',     type=str,   default='adam')

# SageMaker parameters
parser.add_argument('--model_dir',        type=str)
parser.add_argument('--model_output_dir', type=str,   default=os.environ['SM_MODEL_DIR'])
parser.add_argument('--output_data_dir',  type=str,   default=os.environ['SM_OUTPUT_DATA_DIR'])

# Data directories and other options
parser.add_argument('--gpu-count',        type=int,   default=os.environ['SM_NUM_GPUS'])
parser.add_argument('--train',            type=str,   default=os.environ['SM_CHANNEL_TRAIN'])
parser.add_argument('--validation',       type=str,   default=os.environ['SM_CHANNEL_VALIDATION'])
parser.add_argument('--eval',             type=str,   default=os.environ['SM_CHANNEL_EVAL'])

args = parser.parse_args()
```

## SageMaker support for Tensorboard
SageMaker supports real-time Tensorboard monitoring of your training jobs by storing logs in S3 using a Keras callback as follows:

```
class Sync2S3(tf.keras.callbacks.Callback):
def __init__(self, logdir, s3logdir):
    super(Sync2S3, self).__init__()
    self.logdir = logdir
    self.s3logdir = s3logdir

def on_epoch_end(self, batch, logs={}):
    os.system('aws s3 sync '+self.logdir+' '+self.s3logdir)
    # ' >/dev/null 2>&1'
```